<center> <font size =6> 1999 KDD Cup </font></center>

Load the **train** data. In order to save computational time and avoid crashhing my laptop, the 10% subset is used here. The 10% subset is still huge, compared to many datasets. It still has 75 MB, 494021 rows and 42 columns.

In [1]:
import pandas as pd
df=pd.read_csv("KDD_train_data.csv",header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,0,tcp,http,SF,181,5450,0,0,0,0,...,9,1.00,0.00,0.11,0.00,0.0,0.0,0.00,0.00,normal
1,58329,udp,domain_u,SF,42,44,0,0,0,0,...,224,0.88,0.01,0.04,0.00,0.0,0.0,0.00,0.00,normal
2,42448,tcp,ftp_data,RSTR,1,0,0,0,0,0,...,86,0.52,0.14,0.25,0.02,0.0,0.0,0.25,0.01,portsweep
3,42088,tcp,private,RSTR,1,0,0,0,0,0,...,1,0.01,0.19,0.37,0.00,0.0,0.0,0.37,1.00,portsweep
4,41065,tcp,http,RSTR,1,0,0,0,0,0,...,2,0.01,0.32,0.62,0.00,0.0,0.0,0.62,1.00,portsweep


In [2]:
df.shape

(494021, 42)

Separate the target variable from the predictor variables.

In [3]:
y=pd.DataFrame(df.iloc[:,41]).rename(columns={41:"class"})
y

,class
0,normal
1,normal
2,portsweep
3,portsweep
4,portsweep
...,...
494016,normal
494017,normal
494018,normal
494019,normal


Count the instances that belong to each class. The purpose is to examine whether the train data is imbalanced and whether re-balancing is needed.

In [4]:
y.groupby(["class"])["class"].count()

class
back                 2203
buffer_overflow        30
ftp_write               8
guess_passwd           53
imap                   12
ipsweep              1247
land                   21
loadmodule              9
multihop                7
neptune            107201
nmap                  231
normal              97278
perl                    3
phf                     4
pod                   264
portsweep            1040
rootkit                10
satan                1589
smurf              280790
spy                     2
teardrop              979
warezclient          1020
warezmaster            20
Name: class, dtype: int64

The train data is clearly imbalanced, with some classes over-represented and some classes under-represented. Without re-balancing, any model built will highly likely be oversensitive to certain classes and not sensitive to some other classes at all. However, those under-represented classes, which are network intrusion types, might just be as harmful as the over-represented classes. We need the model to be sensitive to all types of intrusions. Re-balancing is needed.

In [5]:
ftp_write = df[df.iloc[:,41]=="ftp_write"]
ftp_write = pd.concat([ftp_write ]*200,axis=0)
ftp_write 

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
4869,134,tcp,login,SF,100,39445,0,0,2,0,...,1,0.5,1.0,0.5,0.00,0.0,0.0,0.0,0.0,ftp_write
4978,67,tcp,login,SF,157,2703,0,0,1,0,...,1,0.5,1.0,0.5,0.00,0.0,0.0,0.0,0.0,ftp_write
5220,32,tcp,ftp,SF,104,449,0,0,0,2,...,1,1.0,0.0,1.0,0.00,0.0,0.0,0.0,0.0,ftp_write
5456,26,tcp,ftp,SF,116,451,0,0,0,2,...,1,1.0,0.0,1.0,0.00,0.0,0.0,0.0,0.0,ftp_write
52214,0,tcp,ftp_data,SF,613,0,0,0,0,0,...,84,1.0,0.0,1.0,0.02,0.0,0.0,0.0,0.0,ftp_write
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5456,26,tcp,ftp,SF,116,451,0,0,0,2,...,1,1.0,0.0,1.0,0.00,0.0,0.0,0.0,0.0,ftp_write
52214,0,tcp,ftp_data,SF,613,0,0,0,0,0,...,84,1.0,0.0,1.0,0.02,0.0,0.0,0.0,0.0,ftp_write
52215,0,tcp,ftp_data,SF,0,5,0,0,0,0,...,85,1.0,0.0,1.0,0.02,0.0,0.0,0.0,0.0,ftp_write
63624,0,tcp,ftp_data,SF,676,0,0,0,0,0,...,4,1.0,0.0,1.0,0.50,0.0,0.0,0.0,0.0,ftp_write


In [6]:
loadmodule=df[df.iloc[:,41]=="loadmodule"]
loadmodule = pd.concat([loadmodule]*200,axis=0)
loadmodule

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
4905,103,tcp,telnet,SF,302,8876,0,0,0,2,...,1,1.00,0.00,1.00,0.00,0.0,0.0,0.0,0.0,loadmodule
4947,85,tcp,telnet,SF,277,693,0,0,0,0,...,1,1.00,0.00,1.00,0.00,0.0,0.0,0.0,0.0,loadmodule
4955,79,tcp,telnet,SF,281,1301,0,0,0,2,...,10,1.00,0.00,1.00,0.30,0.0,0.0,0.0,0.1,loadmodule
5248,31,tcp,telnet,SF,142,1278,0,0,0,0,...,3,0.60,0.60,0.20,0.00,0.0,0.0,0.0,0.0,loadmodule
5731,21,tcp,telnet,SF,135,1290,0,0,0,0,...,4,0.67,0.50,0.17,0.00,0.0,0.0,0.0,0.0,loadmodule
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5731,21,tcp,telnet,SF,135,1290,0,0,0,0,...,4,0.67,0.50,0.17,0.00,0.0,0.0,0.0,0.0,loadmodule
7146,7,tcp,ftp,SF,230,644,0,0,0,4,...,1,0.25,0.75,0.25,0.00,0.0,0.0,0.0,0.0,loadmodule
154729,0,tcp,ftp_data,SF,0,5921,0,0,0,0,...,3,1.00,0.00,1.00,0.67,0.0,0.0,0.0,0.0,loadmodule
154730,0,tcp,ftp_data,SF,0,5014,0,0,0,0,...,4,1.00,0.00,1.00,0.50,0.0,0.0,0.0,0.0,loadmodule


In [7]:
multihop =df[df.iloc[:,41]=="multihop"]
multihop = pd.concat([multihop ]*200,axis=0)
multihop 

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
4513,718,tcp,telnet,SF,1412,25260,0,0,0,15,...,1,1.00,0.00,1.0,0.0,0.0,0.0,0.00,0.0,multihop
4797,198,tcp,telnet,SF,562,9139,0,0,0,3,...,1,1.00,0.00,1.0,0.0,0.0,0.0,0.00,0.0,multihop
4798,192,tcp,ftp,SF,119,426,0,0,0,2,...,1,0.00,0.01,0.0,0.0,0.0,0.0,0.04,0.0,multihop
4813,179,tcp,ftp,SF,87,319,0,0,0,1,...,2,0.01,0.01,0.0,0.0,0.0,0.0,0.04,0.0,multihop
10649,1,tcp,ftp_data,SF,0,988002,0,0,0,0,...,3,1.00,0.00,1.0,0.0,0.0,0.0,0.00,0.0,multihop
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,192,tcp,ftp,SF,119,426,0,0,0,2,...,1,0.00,0.01,0.0,0.0,0.0,0.0,0.04,0.0,multihop
4813,179,tcp,ftp,SF,87,319,0,0,0,1,...,2,0.01,0.01,0.0,0.0,0.0,0.0,0.04,0.0,multihop
10649,1,tcp,ftp_data,SF,0,988002,0,0,0,0,...,3,1.00,0.00,1.0,0.0,0.0,0.0,0.00,0.0,multihop
63618,0,tcp,ftp_data,SF,866,0,0,0,0,0,...,1,1.00,0.00,1.0,0.0,0.0,0.0,0.00,0.0,multihop


In [8]:
perl=df[df.iloc[:,41]=="perl"]
perl = pd.concat([perl]*200,axis=0)
perl

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
5032,54,tcp,telnet,SF,260,2635,0,0,0,0,...,1,0.00,0.01,0.00,0.0,0.0,0.0,0.00,0.0,perl
5089,45,tcp,telnet,SF,268,2364,0,0,0,0,...,2,0.01,0.02,0.00,0.0,0.0,0.0,0.69,0.0,perl
5486,25,tcp,telnet,SF,269,2333,0,0,0,0,...,2,0.03,0.06,0.01,0.0,0.0,0.0,0.00,0.0,perl
5032,54,tcp,telnet,SF,260,2635,0,0,0,0,...,1,0.00,0.01,0.00,0.0,0.0,0.0,0.00,0.0,perl
5089,45,tcp,telnet,SF,268,2364,0,0,0,0,...,2,0.01,0.02,0.00,0.0,0.0,0.0,0.69,0.0,perl
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5089,45,tcp,telnet,SF,268,2364,0,0,0,0,...,2,0.01,0.02,0.00,0.0,0.0,0.0,0.69,0.0,perl
5486,25,tcp,telnet,SF,269,2333,0,0,0,0,...,2,0.03,0.06,0.01,0.0,0.0,0.0,0.00,0.0,perl
5032,54,tcp,telnet,SF,260,2635,0,0,0,0,...,1,0.00,0.01,0.00,0.0,0.0,0.0,0.00,0.0,perl
5089,45,tcp,telnet,SF,268,2364,0,0,0,0,...,2,0.01,0.02,0.00,0.0,0.0,0.0,0.69,0.0,perl


In [9]:
phf=df[df.iloc[:,41]=="phf"]
phf = pd.concat([phf]*200,axis=0)
phf

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
6501,12,tcp,http,SF,51,8127,0,0,0,2,...,246,0.96,0.01,0.0,0.0,0.0,0.0,0.0,0.0,phf
7371,6,tcp,http,SF,51,8127,0,0,0,2,...,249,0.98,0.01,0.0,0.0,0.0,0.0,0.0,0.0,phf
52954,0,tcp,http,SF,51,8127,0,0,0,2,...,255,1.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,phf
95925,0,tcp,http,SF,51,8127,0,0,0,2,...,241,0.95,0.01,0.0,0.0,0.0,0.0,0.0,0.0,phf
6501,12,tcp,http,SF,51,8127,0,0,0,2,...,246,0.96,0.01,0.0,0.0,0.0,0.0,0.0,0.0,phf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95925,0,tcp,http,SF,51,8127,0,0,0,2,...,241,0.95,0.01,0.0,0.0,0.0,0.0,0.0,0.0,phf
6501,12,tcp,http,SF,51,8127,0,0,0,2,...,246,0.96,0.01,0.0,0.0,0.0,0.0,0.0,0.0,phf
7371,6,tcp,http,SF,51,8127,0,0,0,2,...,249,0.98,0.01,0.0,0.0,0.0,0.0,0.0,0.0,phf
52954,0,tcp,http,SF,51,8127,0,0,0,2,...,255,1.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,phf


In [10]:
spy=df[df.iloc[:,41]=="spy"]
spy = pd.concat([spy]*200,axis=0)
spy

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
4676,337,tcp,telnet,SF,237,1540,0,0,0,0,...,47,0.18,0.02,0.0,0.0,0.22,0.32,0.0,0.0,spy
4702,299,tcp,telnet,SF,112,847,0,0,0,0,...,48,0.19,0.02,0.0,0.0,0.22,0.31,0.0,0.0,spy
4676,337,tcp,telnet,SF,237,1540,0,0,0,0,...,47,0.18,0.02,0.0,0.0,0.22,0.32,0.0,0.0,spy
4702,299,tcp,telnet,SF,112,847,0,0,0,0,...,48,0.19,0.02,0.0,0.0,0.22,0.31,0.0,0.0,spy
4676,337,tcp,telnet,SF,237,1540,0,0,0,0,...,47,0.18,0.02,0.0,0.0,0.22,0.32,0.0,0.0,spy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4702,299,tcp,telnet,SF,112,847,0,0,0,0,...,48,0.19,0.02,0.0,0.0,0.22,0.31,0.0,0.0,spy
4676,337,tcp,telnet,SF,237,1540,0,0,0,0,...,47,0.18,0.02,0.0,0.0,0.22,0.32,0.0,0.0,spy
4702,299,tcp,telnet,SF,112,847,0,0,0,0,...,48,0.19,0.02,0.0,0.0,0.22,0.31,0.0,0.0,spy
4676,337,tcp,telnet,SF,237,1540,0,0,0,0,...,47,0.18,0.02,0.0,0.0,0.22,0.32,0.0,0.0,spy


In [11]:
buffer_overflow =df[df.iloc[:,41]=="buffer_overflow"]
buffer_overflow  = pd.concat([buffer_overflow ]*50,axis=0)
buffer_overflow 

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
4683,321,tcp,telnet,RSTO,1506,1887,0,0,0,0,...,6,1.0,0.0,0.17,0.00,0.0,0.0,0.17,0.17,buffer_overflow
4694,305,tcp,telnet,SF,1735,2766,0,0,0,3,...,4,1.0,0.0,0.50,0.50,0.0,0.0,0.00,0.00,buffer_overflow
4712,290,tcp,telnet,SF,415,70529,0,0,0,3,...,1,1.0,0.0,1.00,0.00,0.0,0.0,0.00,0.00,buffer_overflow
4806,184,tcp,telnet,SF,1511,2957,0,0,0,3,...,3,1.0,0.0,1.00,0.67,0.0,0.0,0.00,0.00,buffer_overflow
4814,179,tcp,telnet,SF,1559,2855,0,0,0,3,...,2,1.0,0.0,0.50,0.00,0.0,0.0,0.00,0.00,buffer_overflow
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155923,0,tcp,ftp_data,SF,0,2072,0,0,0,1,...,4,1.0,0.0,1.00,0.00,0.0,0.0,0.00,0.00,buffer_overflow
456123,0,tcp,ftp_data,SF,0,5696,0,0,0,0,...,81,1.0,0.0,1.00,0.02,0.0,0.0,0.00,0.00,buffer_overflow
456124,0,tcp,ftp_data,SF,0,5928,0,0,0,0,...,82,1.0,0.0,1.00,0.02,0.0,0.0,0.00,0.00,buffer_overflow
456125,0,tcp,ftp_data,SF,0,5020,0,0,0,0,...,83,1.0,0.0,1.00,0.02,0.0,0.0,0.00,0.00,buffer_overflow


In [12]:
guess_passwd =df[df.iloc[:,41]=="guess_passwd"]
guess_passwd = pd.concat([guess_passwd]*50,axis=0)
guess_passwd

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
5011,60,tcp,telnet,S3,125,179,0,0,0,1,...,1,1.0,0.0,1.00,0.0,1.00,1.00,0.00,0.00,guess_passwd
5012,60,tcp,telnet,S3,126,179,0,0,0,1,...,23,1.0,0.0,0.04,0.0,0.09,0.09,0.91,0.91,guess_passwd
5582,23,tcp,telnet,SF,104,276,0,0,0,0,...,2,1.0,0.0,1.00,1.0,0.00,0.00,0.00,0.00,guess_passwd
10240,1,tcp,telnet,SF,123,2053,0,0,0,1,...,52,1.0,0.0,0.02,0.0,0.04,0.04,0.94,0.94,guess_passwd
34480,0,tcp,telnet,RSTO,125,179,0,0,0,1,...,2,1.0,0.0,0.50,0.0,0.50,0.50,0.50,0.50,guess_passwd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34524,0,tcp,telnet,RSTO,126,179,0,0,0,1,...,47,1.0,0.0,0.02,0.0,0.04,0.04,0.96,0.96,guess_passwd
34525,0,tcp,telnet,RSTO,126,179,0,0,0,1,...,48,1.0,0.0,0.02,0.0,0.04,0.04,0.96,0.96,guess_passwd
34526,0,tcp,telnet,RSTO,126,179,0,0,0,1,...,49,1.0,0.0,0.02,0.0,0.04,0.04,0.96,0.96,guess_passwd
34527,0,tcp,telnet,RSTO,126,179,0,0,0,1,...,50,1.0,0.0,0.02,0.0,0.04,0.04,0.96,0.96,guess_passwd


In [13]:
imap =df[df.iloc[:,41]=="imap"]
imap = pd.concat([imap]*50,axis=0)
imap

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
5112,41,tcp,imap4,SF,1334,162,0,0,0,0,...,5,1.0,0.00,0.20,0.0,0.40,0.40,0.00,0.0,imap
5240,31,tcp,imap4,SF,1345,10036,0,0,0,0,...,1,0.0,0.01,0.00,0.0,0.00,0.00,0.03,0.0,imap
86448,0,tcp,imap4,SH,0,0,0,0,0,0,...,1,1.0,0.00,1.00,0.0,1.00,1.00,0.00,0.0,imap
86449,0,tcp,imap4,SH,0,0,0,0,0,0,...,2,1.0,0.00,1.00,0.0,1.00,1.00,0.00,0.0,imap
86450,0,tcp,imap4,SF,0,0,0,0,0,0,...,3,1.0,0.00,1.00,0.0,0.67,0.67,0.00,0.0,imap
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86453,0,tcp,imap4,SH,0,0,0,0,0,0,...,7,1.0,0.00,0.71,0.0,0.57,0.57,0.00,0.0,imap
86454,0,tcp,imap4,SH,0,0,0,0,0,0,...,8,1.0,0.00,0.75,0.0,0.62,0.62,0.00,0.0,imap
86455,0,tcp,imap4,SF,0,0,0,0,0,0,...,9,1.0,0.00,0.78,0.0,0.56,0.56,0.00,0.0,imap
86456,0,tcp,imap4,SF,0,0,0,0,0,2,...,10,1.0,0.00,0.80,0.0,0.50,0.50,0.00,0.0,imap


In [14]:
land =df[df.iloc[:,41]=="land"]
land = pd.concat([land]*50,axis=0)
land

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
43189,0,tcp,finger,S0,0,0,1,0,0,0,...,8,1.00,0.00,1.00,0.38,1.00,0.12,0.00,0.0,land
86302,0,tcp,finger,S0,0,0,1,0,0,0,...,1,1.00,0.00,1.00,0.00,1.00,1.00,0.00,0.0,land
86370,0,tcp,finger,S0,0,0,1,0,0,0,...,6,1.00,0.00,1.00,0.33,1.00,0.17,0.00,0.0,land
86458,0,tcp,finger,S0,0,0,1,0,0,0,...,9,0.08,0.17,0.08,0.33,0.58,0.11,0.00,0.0,land
86612,0,tcp,finger,S0,0,0,1,0,0,0,...,1,1.00,0.00,1.00,0.00,1.00,1.00,0.00,0.0,land
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88401,0,tcp,finger,S0,0,0,1,0,0,0,...,6,1.00,0.00,1.00,1.00,1.00,0.83,0.00,0.0,land
354676,0,tcp,finger,S0,0,0,1,0,0,0,...,6,1.00,0.00,1.00,0.50,1.00,0.17,0.00,0.0,land
458807,0,tcp,telnet,S0,0,0,1,0,0,0,...,1,1.00,0.00,1.00,0.00,1.00,1.00,0.00,0.0,land
485556,0,tcp,finger,S0,0,0,1,0,0,0,...,1,0.07,0.20,0.07,0.00,0.07,1.00,0.07,0.0,land


In [15]:
rootkit =df[df.iloc[:,41]=="rootkit"]
rootkit = pd.concat([rootkit]*50,axis=0)
rootkit

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
4516,708,tcp,telnet,SF,1727,24080,0,0,0,0,...,3,0.01,0.02,0.0,0.0,0.0,0.00,0.00,0.00,rootkit
4916,98,tcp,telnet,SF,621,8356,0,0,1,1,...,4,0.02,0.02,0.0,0.0,0.0,0.00,0.00,0.00,rootkit
5008,61,tcp,telnet,SF,294,3929,0,0,0,0,...,4,0.02,0.02,0.0,0.0,0.0,0.25,0.73,0.25,rootkit
5014,60,tcp,telnet,SF,86,183,0,0,0,0,...,1,0.00,0.02,0.0,0.0,0.0,0.00,0.00,0.00,rootkit
5015,60,tcp,telnet,SF,90,233,0,0,0,0,...,2,0.01,0.02,0.0,0.0,0.0,0.00,0.00,0.00,rootkit
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5728,21,tcp,ftp,SF,89,345,0,0,0,1,...,1,0.00,0.02,0.0,0.0,0.0,0.00,0.00,0.00,rootkit
150774,0,tcp,ftp_data,SF,0,5636,0,0,0,0,...,41,1.00,0.00,1.0,0.1,0.0,0.00,0.00,0.00,rootkit
402041,0,udp,other,SF,32,0,0,0,0,0,...,1,0.00,0.02,0.0,0.0,0.0,0.00,0.00,0.00,rootkit
456193,0,udp,other,SF,4,4,0,0,0,0,...,1,1.00,0.00,1.0,0.0,0.0,0.00,0.00,0.00,rootkit


In [16]:
warezmaster =df[df.iloc[:,41]=="warezmaster"]
warezmaster = pd.concat([warezmaster]*50,axis=0)
warezmaster 

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
4840,156,tcp,ftp,SF,950,2551,0,0,0,18,...,1,0.0,0.03,0.0,0.0,0.01,0.0,0.07,0.0,warezmaster
6724,10,tcp,ftp_data,SF,0,5155468,0,0,0,0,...,4,1.0,0.00,1.0,0.0,0.00,0.0,0.00,0.0,warezmaster
6725,10,tcp,ftp_data,SF,0,5151385,0,0,0,0,...,5,1.0,0.00,1.0,0.0,0.00,0.0,0.00,0.0,warezmaster
6726,10,tcp,ftp_data,SF,0,5150823,0,0,0,0,...,7,1.0,0.00,1.0,0.0,0.00,0.0,0.00,0.0,warezmaster
6727,10,tcp,ftp_data,SF,0,5150877,0,0,0,0,...,8,1.0,0.00,1.0,0.0,0.00,0.0,0.00,0.0,warezmaster
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6877,9,tcp,ftp_data,SF,0,5150938,0,0,0,0,...,15,1.0,0.00,1.0,0.0,0.00,0.0,0.00,0.0,warezmaster
9225,2,tcp,ftp_data,SF,0,1159100,0,0,0,0,...,18,1.0,0.00,1.0,0.0,0.00,0.0,0.00,0.0,warezmaster
84504,0,tcp,ftp,SF,36,197,0,0,0,0,...,1,0.0,0.05,0.0,0.0,0.39,0.0,0.05,0.0,warezmaster
87200,0,tcp,ftp_data,SF,0,7181,0,0,0,0,...,1,1.0,0.00,1.0,0.0,0.00,0.0,0.00,0.0,warezmaster


In [17]:
back=df[df.iloc[:,41]=="back"]
back = pd.concat([back]*10,axis=0)
back

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
6122,14,tcp,http,RSTR,42340,1460,0,0,0,0,...,1,1.0,0.0,1.00,0.0,0.00,0.00,1.00,1.00,back
6123,14,tcp,http,RSTR,33580,7300,0,0,0,1,...,4,1.0,0.0,0.25,0.0,0.00,0.00,1.00,1.00,back
6124,14,tcp,http,SF,54540,8314,0,0,0,2,...,18,1.0,0.0,0.06,0.0,0.00,0.00,0.44,0.44,back
6439,13,tcp,http,RSTR,51100,4380,0,0,0,1,...,3,1.0,0.0,0.33,0.0,0.00,0.00,1.00,1.00,back
6440,13,tcp,http,SF,54540,8314,0,0,0,2,...,17,1.0,0.0,0.06,0.0,0.00,0.00,0.47,0.47,back
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
482161,0,tcp,http,SF,54540,8314,0,0,0,2,...,96,1.0,0.0,0.01,0.0,0.01,0.01,0.01,0.01,back
482162,0,tcp,http,SF,54540,8314,0,0,0,2,...,97,1.0,0.0,0.01,0.0,0.01,0.01,0.01,0.01,back
482163,0,tcp,http,SF,54540,8314,0,0,0,2,...,98,1.0,0.0,0.01,0.0,0.01,0.01,0.01,0.01,back
482164,0,tcp,http,SF,54540,8314,0,0,0,2,...,99,1.0,0.0,0.01,0.0,0.01,0.01,0.01,0.01,back


In [18]:
ipsweep=df[df.iloc[:,41]=="ipsweep"]
ipsweep = pd.concat([ipsweep]*10,axis=0)
ipsweep

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
7090,7,tcp,gopher,SF,0,0,0,0,0,0,...,2,0.02,1.0,0.02,1.00,0.0,0.0,0.92,0.0,ipsweep
7265,6,tcp,ssh,RSTO,0,15,0,0,0,0,...,2,0.33,1.0,0.33,1.00,0.0,0.0,1.00,1.0,ipsweep
7266,6,tcp,telnet,SF,0,15,0,0,0,0,...,1,0.25,1.0,0.25,0.00,0.0,0.0,0.75,0.0,ipsweep
7267,6,tcp,domain,SF,0,0,0,0,0,0,...,1,0.03,1.0,0.03,0.00,0.0,0.0,0.88,0.0,ipsweep
7268,6,tcp,gopher,RSTO,0,77,0,0,0,0,...,2,0.02,1.0,0.02,1.00,0.0,0.0,0.90,0.5,ipsweep
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
485551,0,icmp,eco_i,SF,18,0,0,0,0,0,...,255,1.00,0.0,1.00,0.25,0.0,0.0,0.00,0.0,ipsweep
485552,0,icmp,eco_i,SF,18,0,0,0,0,0,...,255,1.00,0.0,1.00,0.25,0.0,0.0,0.00,0.0,ipsweep
485553,0,icmp,eco_i,SF,18,0,0,0,0,0,...,255,1.00,0.0,1.00,0.25,0.0,0.0,0.00,0.0,ipsweep
485554,0,icmp,eco_i,SF,18,0,0,0,0,0,...,255,1.00,0.0,1.00,0.25,0.0,0.0,0.00,0.0,ipsweep


In [19]:
nmap=df[df.iloc[:,41]=="nmap"]
nmap = pd.concat([nmap]*12,axis=0)
nmap

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
53791,0,icmp,eco_i,SF,8,0,0,0,0,0,...,2,0.50,0.75,0.50,0.00,0.00,0.0,0.0,0.0,nmap
53792,0,tcp,private,SH,0,0,0,0,0,0,...,1,0.07,0.71,0.50,0.00,0.50,1.0,0.0,0.0,nmap
53793,0,tcp,private,SH,0,0,0,0,0,0,...,1,0.04,0.75,0.67,0.00,0.67,1.0,0.0,0.0,nmap
53794,0,tcp,telnet,SH,0,0,0,0,0,0,...,1,0.03,0.74,0.76,0.00,0.76,1.0,0.0,0.0,nmap
53795,0,tcp,private,SH,0,0,0,0,0,0,...,1,0.02,0.77,0.82,0.00,0.82,1.0,0.0,0.0,nmap
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86338,0,icmp,eco_i,SF,8,0,0,0,0,0,...,169,1.00,0.00,1.00,0.25,0.00,0.0,0.0,0.0,nmap
86339,0,icmp,eco_i,SF,8,0,0,0,0,0,...,179,1.00,0.00,1.00,0.25,0.00,0.0,0.0,0.0,nmap
86340,0,icmp,eco_i,SF,8,0,0,0,0,0,...,189,1.00,0.00,1.00,0.25,0.00,0.0,0.0,0.0,nmap
86341,0,icmp,eco_i,SF,8,0,0,0,0,0,...,199,1.00,0.00,1.00,0.25,0.00,0.0,0.0,0.0,nmap


In [20]:
pod=df[df.iloc[:,41]=="pod"]
pod = pd.concat([pod]*12,axis=0)
pod

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
27732,0,icmp,ecr_i,SF,1480,0,0,1,0,0,...,1,0.01,0.05,0.01,0.0,0.0,0.0,0.0,0.0,pod
27733,0,icmp,ecr_i,SF,1480,0,0,1,0,0,...,2,0.02,0.05,0.02,0.0,0.0,0.0,0.0,0.0,pod
27734,0,icmp,ecr_i,SF,1480,0,0,1,0,0,...,3,0.03,0.05,0.03,0.0,0.0,0.0,0.0,0.0,pod
27735,0,icmp,ecr_i,SF,1480,0,0,1,0,0,...,4,0.05,0.05,0.05,0.0,0.0,0.0,0.0,0.0,pod
27736,0,icmp,ecr_i,SF,1480,0,0,1,0,0,...,5,0.06,0.04,0.06,0.0,0.0,0.0,0.0,0.0,pod
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459033,0,icmp,ecr_i,SF,1480,0,0,1,0,0,...,20,0.08,0.02,0.08,0.0,0.0,0.0,0.0,0.0,pod
459034,0,icmp,ecr_i,SF,1480,0,0,1,0,0,...,21,0.08,0.02,0.08,0.0,0.0,0.0,0.0,0.0,pod
459035,0,icmp,ecr_i,SF,1480,0,0,1,0,0,...,22,0.09,0.02,0.09,0.0,0.0,0.0,0.0,0.0,pod
460482,0,icmp,tim_i,SF,564,0,0,0,0,0,...,1,1.00,0.00,1.00,0.0,0.0,0.0,0.0,0.0,pod


In [21]:
portsweep=df[df.iloc[:,41]=="portsweep"]
portsweep = pd.concat([portsweep]*10,axis=0)
portsweep

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
2,42448,tcp,ftp_data,RSTR,1,0,0,0,0,0,...,86,0.52,0.14,0.25,0.02,0.0,0.0,0.25,0.01,portsweep
3,42088,tcp,private,RSTR,1,0,0,0,0,0,...,1,0.01,0.19,0.37,0.00,0.0,0.0,0.37,1.00,portsweep
4,41065,tcp,http,RSTR,1,0,0,0,0,0,...,2,0.01,0.32,0.62,0.00,0.0,0.0,0.62,1.00,portsweep
5,40929,tcp,private,RSTR,1,0,0,0,0,0,...,2,0.01,0.34,0.66,0.00,0.0,0.0,0.66,1.00,portsweep
6,40806,tcp,private,RSTR,1,0,0,0,0,0,...,2,0.01,0.36,0.70,0.00,0.0,0.0,0.70,1.00,portsweep
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458778,0,tcp,private,RSTR,0,0,0,0,0,0,...,1,0.00,1.00,1.00,0.00,0.0,0.0,0.99,1.00,portsweep
458779,0,tcp,private,RSTR,0,0,0,0,0,0,...,1,0.00,1.00,1.00,0.00,0.0,0.0,0.99,1.00,portsweep
458780,0,tcp,private,RSTR,0,0,0,0,0,0,...,1,0.00,1.00,1.00,0.00,0.0,0.0,0.99,1.00,portsweep
458781,0,tcp,private,RSTR,0,0,0,0,0,0,...,1,0.00,1.00,1.00,0.00,0.0,0.0,0.99,1.00,portsweep


In [22]:
teardrop=df[df.iloc[:,41]=="teardrop"]
teardrop = pd.concat([teardrop]*10,axis=0)
teardrop

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
31127,0,udp,private,SF,28,0,0,1,0,0,...,1,0.01,0.05,0.01,0.0,0.0,0.0,0.0,0.0,teardrop
31128,0,udp,private,SF,28,0,0,3,0,0,...,2,0.03,0.05,0.03,0.0,0.0,0.0,0.0,0.0,teardrop
31129,0,udp,private,SF,28,0,0,3,0,0,...,3,0.04,0.05,0.04,0.0,0.0,0.0,0.0,0.0,teardrop
31130,0,udp,private,SF,28,0,0,3,0,0,...,4,0.05,0.05,0.05,0.0,0.0,0.0,0.0,0.0,teardrop
31131,0,udp,private,SF,28,0,0,3,0,0,...,5,0.06,0.05,0.06,0.0,0.0,0.0,0.0,0.0,teardrop
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491170,0,udp,private,SF,28,0,0,3,0,0,...,96,0.38,0.01,0.38,0.0,0.0,0.0,0.0,0.0,teardrop
491171,0,udp,private,SF,28,0,0,3,0,0,...,97,0.38,0.01,0.38,0.0,0.0,0.0,0.0,0.0,teardrop
491172,0,udp,private,SF,28,0,0,3,0,0,...,98,0.38,0.01,0.38,0.0,0.0,0.0,0.0,0.0,teardrop
491173,0,udp,private,SF,28,0,0,3,0,0,...,99,0.39,0.01,0.39,0.0,0.0,0.0,0.0,0.0,teardrop


In [23]:
satan=df[df.iloc[:,41]=="satan"]
satan = pd.concat([satan]*10,axis=0)
satan

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
6621,11,tcp,discard,SF,6,0,0,0,0,0,...,1,0.01,0.39,0.01,0.0,0.00,0.0,0.32,0.0,satan
6622,11,tcp,sunrpc,SF,6,0,0,0,0,1,...,1,0.00,1.00,0.00,0.0,0.08,0.0,0.91,0.0,satan
7044,8,tcp,ftp,SF,6,54,0,0,0,0,...,3,0.01,0.43,0.00,0.0,0.00,0.0,0.37,0.0,satan
7337,6,tcp,other,SF,0,0,0,0,0,0,...,1,0.17,0.67,0.17,0.0,0.00,0.0,0.00,0.0,satan
7654,5,tcp,nntp,SF,6,0,0,0,0,0,...,1,0.00,0.33,0.00,0.0,0.00,0.0,0.10,0.0,satan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156655,0,udp,private,SF,1,0,0,0,0,0,...,4,0.02,0.17,1.00,0.0,0.00,0.0,0.00,0.0,satan
156656,0,udp,private,SF,1,0,0,0,0,0,...,3,0.01,0.18,1.00,0.0,0.00,0.0,0.00,0.0,satan
156657,0,udp,private,SF,1,0,0,0,0,0,...,56,0.22,0.18,1.00,0.0,0.00,0.0,0.00,0.0,satan
350637,0,icmp,urp_i,SF,37,0,0,0,0,0,...,1,0.25,0.75,0.50,0.0,0.00,0.0,0.00,0.0,satan


In [24]:
warezclient=df[df.iloc[:,41]=="warezclient"]
warezclient = pd.concat([warezclient]*10,axis=0)
warezclient

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
266,15168,tcp,ftp,SF,365,1202,0,0,0,5,...,3,0.01,0.02,0.0,0.00,0.21,0.0,0.0,0.0,warezclient
267,15163,tcp,ftp,SF,367,1202,0,0,0,6,...,4,0.02,0.02,0.0,0.00,0.20,0.0,0.0,0.0,warezclient
268,15161,tcp,ftp,SF,370,1208,0,0,0,6,...,1,0.00,0.02,0.0,0.00,0.22,0.0,0.0,0.0,warezclient
269,15161,tcp,ftp,SF,368,1202,0,0,0,6,...,5,0.02,0.02,0.0,0.00,0.20,0.0,0.0,0.0,warezclient
270,15160,tcp,ftp,SF,362,1202,0,0,0,6,...,2,0.01,0.02,0.0,0.00,0.21,0.0,0.0,0.0,warezclient
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155860,0,tcp,ftp_data,SF,334,0,0,0,0,0,...,10,1.00,0.00,1.0,0.40,0.00,0.0,0.0,0.0,warezclient
155866,0,tcp,ftp_data,SF,334,0,0,0,0,0,...,11,1.00,0.00,1.0,0.45,0.00,0.0,0.0,0.0,warezclient
155867,0,tcp,ftp_data,SF,334,0,0,0,0,0,...,12,1.00,0.00,1.0,0.42,0.00,0.0,0.0,0.0,warezclient
155875,0,tcp,ftp_data,SF,334,0,0,0,0,0,...,13,1.00,0.00,1.0,0.46,0.00,0.0,0.0,0.0,warezclient


In [25]:
neptune=df[df.iloc[:,41]=="neptune"]
normal=df[df.iloc[:,41]=="normal"]
smurf=df[df.iloc[:,41]=="smurf"]

In [26]:
import numpy as np
print(np.unique(df.iloc[:,41]))

['back' 'buffer_overflow' 'ftp_write' 'guess_passwd' 'imap' 'ipsweep'
 'land' 'loadmodule' 'multihop' 'neptune' 'nmap' 'normal' 'perl' 'phf'
 'pod' 'portsweep' 'rootkit' 'satan' 'smurf' 'spy' 'teardrop'
 'warezclient' 'warezmaster']


In [27]:
classes=[back ,buffer_overflow, ftp_write ,guess_passwd, imap ,ipsweep,
 land, loadmodule, multihop, neptune, nmap, normal ,perl, phf,
 pod, portsweep, rootkit, satan, smurf, spy, teardrop,
 warezclient, warezmaster]
df=pd.concat(classes,axis=0,ignore_index=True)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,14,tcp,http,RSTR,42340,1460,0,0,0,0,...,1,1.0,0.0,1.00,0.0,0.0,0.0,1.00,1.00,back
1,14,tcp,http,RSTR,33580,7300,0,0,0,1,...,4,1.0,0.0,0.25,0.0,0.0,0.0,1.00,1.00,back
2,14,tcp,http,SF,54540,8314,0,0,0,2,...,18,1.0,0.0,0.06,0.0,0.0,0.0,0.44,0.44,back
3,13,tcp,http,RSTR,51100,4380,0,0,0,1,...,3,1.0,0.0,0.33,0.0,0.0,0.0,1.00,1.00,back
4,13,tcp,http,SF,54540,8314,0,0,0,2,...,17,1.0,0.0,0.06,0.0,0.0,0.0,0.47,0.47,back


Size of the re-balanced **train** data.

In [28]:
df.shape

(585889, 42)

Assign integer label to the categorical variables.

In [29]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df_1=pd.DataFrame(df.iloc[:,1])
df_1=pd.DataFrame(le.fit_transform(df_1))
df_1.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/preprocessing/label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,0
0,1
1,1
2,1
3,1
4,1


In [30]:
df_2=pd.DataFrame(df.iloc[:,2])
df_2=pd.DataFrame(le.fit_transform(df_2))
df_2.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/preprocessing/label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,0
0,22
1,22
2,22
3,22
4,22


In [31]:
df_3=pd.DataFrame(df.iloc[:,3])
df_3=pd.DataFrame(le.fit_transform(df_3))
df_3.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/preprocessing/label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,0
0,4
1,4
2,9
3,4
4,9


In [32]:
sub_X=df.drop(columns=[1,2,3,41])
sub_X.head()

,0,4,5,6,7,8,9,10,11,12,...,31,32,33,34,35,36,37,38,39,40
0,14,42340,1460,0,0,0,0,0,1,0,...,1,1,1.0,0.0,1.00,0.0,0.0,0.0,1.00,1.00
1,14,33580,7300,0,0,0,1,0,1,0,...,4,4,1.0,0.0,0.25,0.0,0.0,0.0,1.00,1.00
2,14,54540,8314,0,0,0,2,0,1,1,...,18,18,1.0,0.0,0.06,0.0,0.0,0.0,0.44,0.44
3,13,51100,4380,0,0,0,1,0,1,0,...,3,3,1.0,0.0,0.33,0.0,0.0,0.0,1.00,1.00
4,13,54540,8314,0,0,0,2,0,1,1,...,17,17,1.0,0.0,0.06,0.0,0.0,0.0,0.47,0.47


In [33]:
X=pd.concat([df_1,df_2,df_3,sub_X],axis=1)
X.head()

,0,0,0,0,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
0,1,22,4,14,42340,1460,0,0,0,0,...,1,1,1.0,0.0,1.00,0.0,0.0,0.0,1.00,1.00
1,1,22,4,14,33580,7300,0,0,0,1,...,4,4,1.0,0.0,0.25,0.0,0.0,0.0,1.00,1.00
2,1,22,9,14,54540,8314,0,0,0,2,...,18,18,1.0,0.0,0.06,0.0,0.0,0.0,0.44,0.44
3,1,22,4,13,51100,4380,0,0,0,1,...,3,3,1.0,0.0,0.33,0.0,0.0,0.0,1.00,1.00
4,1,22,9,13,54540,8314,0,0,0,2,...,17,17,1.0,0.0,0.06,0.0,0.0,0.0,0.47,0.47


Standardize the predictor variables.

In [34]:
from sklearn import preprocessing
scaler=preprocessing.StandardScaler()
X=scaler.fit_transform(X)
X=pd.DataFrame(X)
X.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
0,0.765482,-0.180547,-1.531112,-0.057315,0.007872,-0.036196,-0.042392,-0.141665,-0.026224,-0.121876,...,-2.720885,-1.522931,0.643012,-0.325918,0.880277,-0.207479,-0.439294,-0.43219,3.281671,3.243139
1,0.765482,-0.180547,-1.531112,-0.057315,0.004819,-0.004846,-0.042392,-0.141665,-0.026224,0.369831,...,-2.683662,-1.496197,0.643012,-0.325918,-0.675228,-0.207479,-0.439294,-0.43219,3.281671,3.243139
2,0.765482,-0.180547,0.526977,-0.057315,0.012124,0.000597,-0.042392,-0.141665,-0.026224,0.861538,...,-2.509954,-1.371438,0.643012,-0.325918,-1.069289,-0.207479,-0.439294,-0.43219,1.255560,1.248026
3,0.765482,-0.180547,-1.531112,-0.058159,0.010925,-0.020521,-0.042392,-0.141665,-0.026224,0.369831,...,-2.696070,-1.505108,0.643012,-0.325918,-0.509307,-0.207479,-0.439294,-0.43219,3.281671,3.243139
4,0.765482,-0.180547,0.526977,-0.058159,0.012124,0.000597,-0.042392,-0.141665,-0.026224,0.861538,...,-2.522362,-1.380350,0.643012,-0.325918,-1.069289,-0.207479,-0.439294,-0.43219,1.364102,1.354907


In [35]:
y=pd.DataFrame(df.iloc[:,41])
y.head()

,41
0,back
1,back
2,back
3,back
4,back


Build the Random Forest model.

In [36]:
from sklearn.ensemble import RandomForestClassifier
tree = RandomForestClassifier(max_depth=20,class_weight="balanced_subsample",criterion="entropy",random_state=0)
tree

RandomForestClassifier(bootstrap=True, class_weight='balanced_subsample',
            criterion='entropy', max_depth=20, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators='warn', n_jobs=None, oob_score=False,
            random_state=0, verbose=0, warm_start=False)

Train the model.

In [37]:
tree.fit(X,y)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestClassifier(bootstrap=True, class_weight='balanced_subsample',
            criterion='entropy', max_depth=20, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=None, oob_score=False, random_state=0,
            verbose=0, warm_start=False)

Calculate the model's accuracy on train data.

In [38]:
from sklearn.metrics import accuracy_score
y_pred=tree.predict(X)
print("Accuracy on train data is: ",accuracy_score(y,y_pred))

Accuracy on train data is:  0.9999743978808272


Load the **test** data.

In [39]:
df=pd.read_csv("corrected.csv",header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,0,udp,private,SF,105,146,0,0,0,0,...,254,1.0,0.01,0.00,0.0,0.0,0.0,0.0,0.0,normal
1,0,udp,private,SF,105,146,0,0,0,0,...,254,1.0,0.01,0.00,0.0,0.0,0.0,0.0,0.0,normal
2,0,udp,private,SF,105,146,0,0,0,0,...,254,1.0,0.01,0.00,0.0,0.0,0.0,0.0,0.0,normal
3,0,udp,private,SF,105,146,0,0,0,0,...,254,1.0,0.01,0.00,0.0,0.0,0.0,0.0,0.0,snmpgetattack
4,0,udp,private,SF,105,146,0,0,0,0,...,254,1.0,0.01,0.01,0.0,0.0,0.0,0.0,0.0,snmpgetattack


In [40]:
df.shape

(311029, 42)

Assign integer label to categorical variables and standardize all predictor variables.

In [41]:
df_1=pd.DataFrame(df.iloc[:,1])
df_1=pd.DataFrame(le.fit_transform(df_1))

df_2=pd.DataFrame(df.iloc[:,2])
df_2=pd.DataFrame(le.fit_transform(df_2))

df_3=pd.DataFrame(df.iloc[:,3])
df_3=pd.DataFrame(le.fit_transform(df_3))

sub_X=df.drop(columns=[1,2,3,41])

X=pd.concat([df_1,df_2,df_3,sub_X],axis=1)
X.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/preprocessing/label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,0,0,0,0,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
0,2,46,9,0,105,146,0,0,0,0,...,255,254,1.0,0.01,0.00,0.0,0.0,0.0,0.0,0.0
1,2,46,9,0,105,146,0,0,0,0,...,255,254,1.0,0.01,0.00,0.0,0.0,0.0,0.0,0.0
2,2,46,9,0,105,146,0,0,0,0,...,255,254,1.0,0.01,0.00,0.0,0.0,0.0,0.0,0.0
3,2,46,9,0,105,146,0,0,0,0,...,255,254,1.0,0.01,0.00,0.0,0.0,0.0,0.0,0.0
4,2,46,9,0,105,146,0,0,0,0,...,255,254,1.0,0.01,0.01,0.0,0.0,0.0,0.0,0.0


In [42]:
X=scaler.fit_transform(X)
X=pd.DataFrame(X)
X.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
0,2.232618,1.428547,0.47954,-0.043918,-0.012743,-0.037344,-0.005379,-0.018876,-0.005238,-0.047032,...,0.323695,0.546387,0.533484,-0.155751,-1.113742,-0.127644,-0.254066,-0.252326,-0.414249,-0.408842
1,2.232618,1.428547,0.47954,-0.043918,-0.012743,-0.037344,-0.005379,-0.018876,-0.005238,-0.047032,...,0.323695,0.546387,0.533484,-0.155751,-1.113742,-0.127644,-0.254066,-0.252326,-0.414249,-0.408842
2,2.232618,1.428547,0.47954,-0.043918,-0.012743,-0.037344,-0.005379,-0.018876,-0.005238,-0.047032,...,0.323695,0.546387,0.533484,-0.155751,-1.113742,-0.127644,-0.254066,-0.252326,-0.414249,-0.408842
3,2.232618,1.428547,0.47954,-0.043918,-0.012743,-0.037344,-0.005379,-0.018876,-0.005238,-0.047032,...,0.323695,0.546387,0.533484,-0.155751,-1.113742,-0.127644,-0.254066,-0.252326,-0.414249,-0.408842
4,2.232618,1.428547,0.47954,-0.043918,-0.012743,-0.037344,-0.005379,-0.018876,-0.005238,-0.047032,...,0.323695,0.546387,0.533484,-0.155751,-1.093415,-0.127644,-0.254066,-0.252326,-0.414249,-0.408842


In [43]:
X.shape

(311029, 41)

Test the model and output the accuracy on test data.

In [51]:
y_pred=tree.predict(X)
print("Accuracy on test data is: ",accuracy_score(y,y_pred))

Accuracy on test data is:  0.9122364795565687


**91.22% accuracy on test data.That is my solution to the 1999 KDD Cup.**